# RAG vs. GraphRAG: Investigative Intelligence Comparison

## Overview

This notebook provides a rigorous, side-by-side comparison of **Standard RAG (Vector-based)** and **GraphRAG (Graph-based)**, focusing on how GraphRAG creates a "Chain of Evidence" that Vector RAG cannot see.

### The Challenge: Navigating Fragmentation

In intelligence work, facts are scattered across reports. Vector search often fails to bridge "semantic gaps"—logical connections between entities that are not physically co-located in text. 

We will demonstrate how GraphRAG creates a **"Chain of Evidence"** that Vector RAG cannot see.

### Framework: Semantica

We use the [Semantica](https://github.com/Hawksight-AI/semantica) framework to orchestrate common intelligence tasks like entity resolution, conflict detection, and graph-based reasoning.


In [1]:
import os
from pathlib import Path

from dotenv import load_dotenv
load_dotenv("D:\Work\semantica\.env")

True

In [2]:
# Environment Setup
import os

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY', '')

# Install Semantica and all required dependencies
#%pip install -qU semantica networkx matplotlib plotly pandas faiss-cpu beautifulsoup4 groq sentence-transformers


In [3]:
os.getenv('GROQ_API_KEY', '')

'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'

---

## Setup & Configuration

Configure the environment and import necessary modules for both RAG approaches.


### Import Required Modules

Import modules for data ingestion, processing, and RAG approaches.


In [4]:
# Core imports will be added in cells where they're first used
import os
print("Environment ready. Imports will be added as needed in subsequent cells.")


Environment ready. Imports will be added as needed in subsequent cells.


### Configure API Keys

Set up API keys for LLM providers. In production, use environment variables.


In [5]:
# Set up API keys
# Note: In production, use environment variables: export GROQ_API_KEY="your-key"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "")

print("API keys configured.")


API keys configured.


---

## Section 1: Domain Acquisition

Build a knowledge base from real-world intelligence sources to demonstrate Vector RAG vs GraphRAG differences.

### Data Sources

- **RSS Feeds**: BBC, Al Jazeera, Reuters
- **Web Pages**: Wikipedia articles on intelligence analysis
- **Real-World Data**: Live feeds with diverse perspectives and realistic complexity


In [6]:
from semantica.ingest import WebIngestor, FeedIngestor
from semantica.split import EntityAwareChunker
from semantica.normalize import TextNormalizer

In [7]:
normalizer = TextNormalizer()

In [8]:
all_content = []

feeds = [
    "http://feeds.bbci.co.uk/news/world/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml",
    "https://news.google.com/rss/search?q=site%3Areuters.com&hl=en-US&gl=US&ceid=US%3Aen"
]

In [9]:

feed_ingestor = FeedIngestor()
for f in feeds:
    try:
        data = feed_ingestor.ingest_feed(f)
        items = data.items[:10]
        for item in items:
            text = item.content or item.description or item.title
            if text:
                all_content.append(text)
    except Exception as e:
        print(f"Warning: Failed to ingest feed {f}: {e}")

web_urls = ["https://en.wikipedia.org/wiki/Intelligence_analysis"]
web_ingestor = WebIngestor()
for url in web_urls:
    try:
        content = web_ingestor.ingest_url(url)
        if content and content.text:
            all_content.append(content.text)
    except Exception as e:
        print(f"Warning: Failed to ingest URL {url}: {e}")

clean_docs = [normalizer.normalize(text) for text in all_content if len(text) > 100]
print(f"Intelligence Knowledge Hub Populated with {len(clean_docs)} reports.")


Status,Action,Module,Submodule,Progress,ETA,Rate,Time,Extracted
✅,Semantica is deduplicating,🔄 deduplication,SimilarityCalculator,100.0% (5/5),-,0.0/s,0.02s,-
✅,Semantica is deduplicating,🔄 deduplication,EntityMerger,100.0% (1/1),-,0.0/s,0.06s,-
✅,Semantica is deduplicating,🔄 deduplication,MergeStrategyManager,100.0% (4/4),-,0.0/s,0.01s,-
✅,Semantica is resolving,⚠️ conflicts,ConflictDetector,100.0% (0/24),-,-,0.09s,-
❌,Semantica is embedding,💾 embeddings,TextEmbedder,-,-,-,0.02s,-
✅,Semantica is processing,🔗 context,ContextRetriever,100.0%,-,-,1.26s,-
✅,Semantica is processing,🔗 context,AgentMemory,100.0%,-,-,0.04s,-
✅,Semantica is visualizing,📈 visualization,KGVisualizer,100.0%,-,-,0.02s,-
✅,Semantica is building,🧠 kg,CentralityCalculator,100.0%,-,-,0.01s,-
✅,Semantica is building,🧠 kg,CommunityDetector,100.0%,-,-,0.01s,-


🔄 Semantica is ingesting: Feed: http://feeds.bbci.co.uk/news/world/rss.xml 📥 ingest FeedIngestor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is ingesting: Ingested 31 items 📥 ingest FeedIngestor |███████████████| 100.0% ETA: - Rate: - Time: 0.42s Extracted: -✅ Semantica is ingesting: Ingested 25 items 📥 ingest FeedIngestor |███████████████| 100.0% ETA: - Rate: - Time: 0.24s Extracted: -✅ Semantica is ingesting: Ingested 100 items 📥 ingest FeedIngestor |███████████████| 100.0% ETA: - Rate: - Time: 1.00s Extracted: -

URL https://en.wikipedia.org/wiki/Intelligence_analysis blocked by robots.txt


❌ Semantica is ingesting: URL blocked by robots.txt: https://en.wikipedia.org/wiki/Intelligence_analysis 📥 ingest WebIngestor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.18s Extracted: -Warning: Failed to ingest URL https://en.wikipedia.org/wiki/Intelligence_analysis: URL blocked by robots.txt: https://en.wikipedia.org/wiki/Intelligence_analysis
🔄 Normalizing text 🔧 normalize TextNormalizer |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.02s Extracted: -Intelligence Knowledge Hub Populated with 22 reports.


### Ingest RSS Feeds

RSS feeds provide structured, regularly updated content from news sources.


In [10]:
# Define RSS feed URLs
feeds = [
    "http://feeds.bbci.co.uk/news/world/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml",
    "https://news.google.com/rss/search?q=site%3Areuters.com&hl=en-US&gl=US&ceid=US%3Aen"
]

print("Ingesting from RSS feeds...")
for f in feeds:
    try:
        print(f"  Processing: {f}")
        data = feed_ingestor.ingest_feed(f)
        items = data.items[:10]  # Limit to 10 items per feed
        
        for item in items:
            text = item.content or item.description or item.title
            if text:
                all_content.append(text)
        
        print(f"    Successfully ingested {len(items)} items")
    except Exception as e:
        print(f"    Warning: Failed to ingest feed {f}: {e}")

print(f"\nTotal feed items ingested: {len([c for c in all_content if c])}")


Ingesting from RSS feeds...
  Processing: http://feeds.bbci.co.uk/news/world/rss.xml
🔄 Semantica is ingesting: Feed: http://feeds.bbci.co.uk/news/world/rss.xml 📥 ingest FeedIngestor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is ingesting: Ingested 31 items 📥 ingest FeedIngestor |███████████████| 100.0% ETA: - Rate: - Time: 0.60s Extracted: -    Successfully ingested 10 items
  Processing: https://www.aljazeera.com/xml/rss/all.xml
✅ Semantica is ingesting: Ingested 25 items 📥 ingest FeedIngestor |███████████████| 100.0% ETA: - Rate: - Time: 0.23s Extracted: -    Successfully ingested 10 items
  Processing: https://news.google.com/rss/search?q=site%3Areuters.com&hl=en-US&gl=US&ceid=US%3Aen
✅ Semantica is ingesting: Ingested 100 items 📥 ingest FeedIngestor |███████████████| 100.0% ETA: - Rate: - Time: 0.29s Extracted: -    Successfully ingested 10 items

Total feed items ingested: 60


In [11]:
all_content

["The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.",
 'The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.',
 "Critics say the proposal could see even younger children becoming offenders as well as breaching children's rights.",
 'Firefighters in Victoria are battling at least six major fires amid a record-breaking heatwave.',
 'Shop owners say the frowning toys - originally made in error - are resonating with young workers.',
 'The African nation argues it is owed payments but Downing Street says it will fight the case.',
 'Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week.',
 'The political thriller has 14 nominations, with Sinners, Hamnet and Marty Supreme also recognised.',
 'Trump has accused Seoul of "not living up" to a trade deal that was reached last year.',
 'The 80-year-old i

### Ingest Web Pages

Web ingestion extracts content from specific web pages, including Wikipedia articles.


In [12]:
# Define web URLs to ingest
web_urls = [
    "https://en.wikipedia.org/wiki/Intelligence_analysis"
]

print("Ingesting from web pages...")
for url in web_urls:
    try:
        print(f"  Processing: {url}")
        content = web_ingestor.ingest_url(url)
        if content and content.text:
            all_content.append(content.text)
            print(f"    Successfully ingested content ({len(content.text)} characters)")
    except Exception as e:
        print(f"    Warning: Failed to ingest URL {url}: {e}")

print(f"\nTotal web pages ingested: {len([c for c in all_content if c])}")


Ingesting from web pages...
  Processing: https://en.wikipedia.org/wiki/Intelligence_analysis
🔄 Semantica is ingesting: URL: https://en.wikipedia.org/wiki/Intelligence_analysis 📥 ingest WebIngestor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -

URL https://en.wikipedia.org/wiki/Intelligence_analysis blocked by robots.txt



Total web pages ingested: 60


In [13]:
all_content

["The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.",
 'The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.',
 "Critics say the proposal could see even younger children becoming offenders as well as breaching children's rights.",
 'Firefighters in Victoria are battling at least six major fires amid a record-breaking heatwave.',
 'Shop owners say the frowning toys - originally made in error - are resonating with young workers.',
 'The African nation argues it is owed payments but Downing Street says it will fight the case.',
 'Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week.',
 'The political thriller has 14 nominations, with Sinners, Hamnet and Marty Supreme also recognised.',
 'Trump has accused Seoul of "not living up" to a trade deal that was reached last year.',
 'The 80-year-old i

### Normalize Content

Text normalization cleans and standardizes all ingested content for consistent processing.


In [14]:
from semantica.normalize import TextNormalizer
# Normalize all ingested content
print("Normalizing content...")
clean_docs = []

for text in all_content:
    if len(text) > 100:  # Filter out very short content
        normalized_text = normalizer.normalize(text)
        clean_docs.append(normalized_text)

print(f"\nIntelligence Knowledge Hub Populated with {len(clean_docs)} reports.")
print(f"  - Total documents: {len(clean_docs)}")
print(f"  - Ready for processing")


Normalizing content...
🔄 Normalizing text 🔧 normalize TextNormalizer |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is normalizing 🔧 normalize TextNormalizer |███████████████| 100.0% ETA: - Rate: - Time: 0.00s Extracted: -🔄 Normalizing text 🔧 normalize TextNormalizer |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.03s Extracted: -
Intelligence Knowledge Hub Populated with 44 reports.
  - Total documents: 44
  - Ready for processing


---

## Section 2: Standard Vector RAG Pipeline

Traditional vector-based RAG using semantic similarity search only.

### How Vector RAG Works

1. Chunk documents
2. Generate embeddings
3. Store vectors in FAISS
4. Query using cosine similarity

### Limitations

- No relationship awareness
- Misses connections between distant facts
- No multi-hop reasoning
- Context fragmentation


In [15]:
from semantica.core import Semantica
from semantica.vector_store import VectorStore
from semantica.split import EntityAwareChunker

v_core = Semantica()
splitter = EntityAwareChunker(chunk_size=600, chunk_overlap=50)
chunks = []
for doc in clean_docs[:10]:
    chunks.extend(splitter.chunk(doc))

vs = VectorStore(backend="faiss", dimension=384)
embeddings = v_core.embedding_generator.generate_embeddings([str(c.text) for c in chunks[:15]])
vs.store_vectors(vectors=embeddings, metadata=[{"content": str(c.text)} for c in chunks[:15]])

print(f"Vector RAG ready with {len(chunks[:15])} encoded fragments.")


🔄 Semantica is splitting: Chunking text with entity awareness ✂️ split EntityAwareChunker |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.01s Extracted: -🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is extracting: Extracted 3 entities using ml 🎯 semantic_extract NERExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.33s Extracted: -🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is extracting: Extracted 3 entities using ml 🎯 semantic_extract NERExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.34s Extracted: -🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is extracting: Extracted 1 entities 🎯 semantic_extract NERE

In [16]:
chunks

[Chunk(text="The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.", start_index=0, end_index=101, metadata={'method': 'entity_aware', 'chunk_size': 101, 'entity_count': 3, 'entities': [Entity(text='US', label='GPE', start_char=4, end_char=6, confidence=1.0, metadata={'extraction_method': 'ml', 'model': 'en_core_web_sm', 'lemma': 'US', 'batch_index': 0}), Entity(text='February', label='DATE', start_char=67, end_char=75, confidence=1.0, metadata={'extraction_method': 'ml', 'model': 'en_core_web_sm', 'lemma': 'February', 'batch_index': 0}), Entity(text='Milan', label='GPE', start_char=87, end_char=92, confidence=1.0, metadata={'extraction_method': 'ml', 'model': 'en_core_web_sm', 'lemma': 'Milan', 'batch_index': 0})]}, id=None),
 Chunk(text='The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.', start_index=0, end_index=115, metadata={'method': 'entity_aware', 'chunk_size': 115,

### Chunk Documents

Split documents into semantic chunks for embedding generation.


In [17]:
# Initialize chunker
splitter = EntityAwareChunker(chunk_size=600, chunk_overlap=50)

# Chunk documents
print("Chunking documents...")
chunks = []

for i, doc in enumerate(clean_docs[:10], 1):
    doc_chunks = splitter.chunk(doc)
    chunks.extend(doc_chunks)
    print(f"  Document {i}: {len(doc_chunks)} chunks created")

print(f"\nTotal chunks created: {len(chunks)}")
print(f"  - Chunk size: 600 characters")
print(f"  - Overlap: 50 characters")


Chunking documents...
🔄 Semantica is splitting: Chunking text with entity awareness ✂️ split EntityAwareChunker |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.01s Extracted: -✅ Semantica is extracting: Extracted 3 entities using ml 🎯 semantic_extract NERExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.36s Extracted: -  Document 1: 1 chunks created
🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.01s Extracted: -✅ Semantica is extracting: Extracted 3 entities using ml 🎯 semantic_extract NERExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.33s Extracted: -  Document 2: 1 chunks created
🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s

### Generate Embeddings

Create vector embeddings for all chunks using the embedding model.


In [18]:
# Generate embeddings for chunks (limit to 15 for demonstration)
print("Generating embeddings...")
chunks_to_embed = chunks[:15]
texts_to_embed = [str(c.text) for c in chunks_to_embed]

embeddings = v_core.embedding_generator.generate_embeddings(texts_to_embed)

print(f"Embeddings generated:")
print(f"  - Total embeddings: {len(embeddings)}")
print(f"  - Embedding dimension: {embeddings.shape[1] if len(embeddings) > 0 else 0}")


Generating embeddings...
Embeddings generated:
  - Total embeddings: 10
  - Embedding dimension: 384


In [19]:
texts_to_embed

["The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.",
 'The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.',
 "Critics say the proposal could see even younger children becoming offenders as well as breaching children's rights.",
 'Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week.',
 'The 80-year-old is accused of crimes of humanity over his bloody war on drugs that left thousands killed.',
 'Former Real Madrid manager Jose Mourinho can stifle his former side in must-win UCL match for his current club, Benfica.',
 'Coco Gauff, who lost to Elina Svitolina in the quarterfinals of the Australian Open, was filmed smashing her racket.',
 'Hamza Al-Rubaie is one of at least 17,000 children to have lost both parents in Israel’s genocidal war on Gaza.',
 'Rial hits record \u200d1,500,000 rials 

### Store Vectors

Store embeddings in FAISS vector store for fast similarity search.


In [20]:
from semantica.vector_store import VectorStore

# Initialize vector store
vs = VectorStore(backend="faiss", dimension=384)

# Prepare metadata
metadata = [{"content": str(c.text)} for c in chunks_to_embed]

# Store vectors
print("Storing vectors in vector store...")
vs.store_vectors(vectors=embeddings, metadata=metadata)

print(f"\nVector RAG ready with {len(chunks_to_embed)} encoded fragments.")
print(f"  - Vector store backend: FAISS")
print(f"  - Ready for semantic similarity search")


Storing vectors in vector store...
🔄 Semantica is indexing: Storing 10 vectors 📊 vector_store VectorStore |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -
Vector RAG ready with 10 encoded fragments.
  - Vector store backend: FAISS
  - Ready for semantic similarity search


---

## Section 3: GraphRAG Pipeline

GraphRAG combines vector search with knowledge graph traversal.

### How GraphRAG Works

1. Extract entities and relationships to build knowledge graph
2. Generate embeddings (same as Vector RAG)
3. Store vectors in database
4. Hybrid query: vector search + graph traversal with hybrid scoring

### Advantages

- Multi-hop reasoning across entities
- Bridges semantic gaps between distant facts
- Automatic context expansion
- Relationship awareness


In [21]:
from semantica.semantic_extract import NERExtractor, RelationExtractor

# Initialize extractors
print("Initializing GraphRAG extractors...")
ner = NERExtractor(method="llm", provider="groq", model="llama-3.1-8b-instant")
rel_ext = RelationExtractor(method="llm", provider="groq", model="llama-3.1-8b-instant")

print("Extractors initialized.")
print("  - NER Extractor: Ready")
print("  - Relation Extractor: Ready")


Initializing GraphRAG extractors...
Extractors initialized.
  - NER Extractor: Ready
  - Relation Extractor: Ready


### Extract Entities and Relationships

Extract structured knowledge from document chunks to build the knowledge graph.


In [22]:
# Container for extraction results
kg_sources = {"entities": [], "relationships": []}

print("Extracting entities and relationships from chunks...")
print(f"Processing {min(10, len(chunks))} chunks...\n")

for i, chunk in enumerate(chunks[:10], 1):
    txt = str(chunk.text)
    try:
        print(f"Chunk {i}:")
        
        # Extract entities
        entities = ner.extract(txt)
        for e in entities:
            kg_sources["entities"].append({
                "name": e.text,
                "type": e.label,
                "id": e.text.lower().replace(' ', '_')
            })
        
        print(f"  Found {len(entities)} entities")
        
        # Extract relationships (requires entities)
        if entities:
            relations = rel_ext.extract(txt, entities=entities)
            for r in relations:
                kg_sources["relationships"].append({
                    "source": r.subject,
                    "target": r.object,
                    "type": r.predicate
                })
            print(f"  Found {len(relations)} relationships")
        else:
            print(f"  No relationships (no entities found)")
        
    except Exception as e:
        print(f"  Warning: Error extracting from chunk: {e}")
        continue

print(f"\nExtraction complete:")
print(f"  - Total entities extracted: {len(kg_sources['entities'])}")
print(f"  - Total relationships extracted: {len(kg_sources['relationships'])}")


Extracting entities and relationships from chunks...
Processing 10 chunks...

Chunk 1:
🔄 Semantica is extracting: Extracting named entities from text 🎯 semantic_extract NERExtractor |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is extracting: Extracted 3 entities using llm 🎯 semantic_extract NERExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.65s Extracted: -  Found 3 entities
✅ Semantica is extracting: Extracted 2 relations using llm 🎯 semantic_extract RelationExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.84s Extracted: -  Found 2 relationships
Chunk 2:
✅ Semantica is extracting: Extracted 3 entities using llm 🎯 semantic_extract NERExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.22s Extracted: -  Found 3 entities
✅ Semantica is extracting: Extracted 2 relations using llm 🎯 semantic_extract RelationExtractor |███████████████| 100.0% ETA: - Rate: - Time: 0.21s Extracted: -  Found 2 relationships
Chunk 3:
✅ Semantica is extractin

### Build Knowledge Graph

Construct the knowledge graph from extracted entities and relationships.


In [23]:
from semantica.kg import GraphBuilder

# Initialize GraphBuilder
gb = GraphBuilder(merge_entities=True)

print("Building knowledge graph...")
kg_data = gb.build(sources=[kg_sources])

print(f"Initial graph:")
print(f"  - Entities: {len(kg_data.get('entities', []))}")
print(f"  - Relationships: {len(kg_data.get('relationships', []))}")


Building knowledge graph...
🔄 Semantica is building: Knowledge graph from 1 source(s) 🧠 kg GraphBuilder |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -🔄 Semantica is building: Processing entities... 33/33 🧠 kg GraphBuilder |███████████████| 100.0% ETA: - Rate: 5886.6/s Time: 0.01s Extracted: -🔄 Semantica is building: Processing relationships... 27/27 🧠 kg GraphBuilder |███████████████| 100.0% ETA: - Rate: 4116.5/s Time: 0.01s Extracted: -Resolving 33 entities...
🔄 Semantica is deduplicating: Comparing pairs in block 'u'... 1/30 🔄 deduplication SimilarityCalculator |░░░░░░░░░░░░░░░| 3.3% ETA: 0.1s Rate: 130.5/s Time: 0.01s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 'u'... 2/30 🔄 deduplication SimilarityCalculator |█░░░░░░░░░░░░░░| 6.7% ETA: 0.2s Rate: 157.3/s Time: 0.01s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 'u'... 3/30 🔄 deduplication SimilarityCalculator |█░░░░░░░░░░░░░░| 10.0% ETA: 0.1s Rate: 183.1/s Time: 0.02s Extr

### Resolve Entities

Entity resolution merges duplicate entities to create a cleaner, more consistent graph.


In [24]:
from semantica.kg import EntityResolver

# Initialize EntityResolver
resolver = EntityResolver(similarity_threshold=0.85)

print("Resolving entities (deduplication)...")
print(f"  Similarity threshold: 0.85 (85%)")

# Resolve entities
kg_data['entities'] = resolver.resolve_entities(kg_data.get('entities', []))

print(f"\nEntity resolution complete:")
print(f"  - Resolved entities: {len(kg_data['entities'])}")


Resolving entities (deduplication)...
  Similarity threshold: 0.85 (85%)
🔄 Semantica is building: Resolving entities 🧠 kg EntityResolver |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 'u'... 1/5 🔄 deduplication SimilarityCalculator |███░░░░░░░░░░░░| 20.0% ETA: 0.0s Rate: 162.5/s Time: 0.01s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 'c'... 2/5 🔄 deduplication SimilarityCalculator |██████░░░░░░░░░| 40.0% ETA: 0.0s Rate: 230.8/s Time: 0.01s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 'b'... 3/5 🔄 deduplication SimilarityCalculator |█████████░░░░░░| 60.0% ETA: 0.0s Rate: 249.2/s Time: 0.01s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 'w'... 4/5 🔄 deduplication SimilarityCalculator |████████████░░░| 80.0% ETA: 0.0s Rate: 275.3/s Time: 0.01s Extracted: -🔄 Semantica is deduplicating: Comparing pairs in block 's'... 5/5 🔄 deduplication SimilarityCalculato

### Initialize AgentContext for Hybrid Retrieval

AgentContext enables hybrid retrieval by combining vector search with graph traversal.


In [25]:
from semantica.context import AgentContext

# Initialize AgentContext with hybrid retrieval
ctx = AgentContext(
    vector_store=vs,                    # Same vector store as Vector RAG
    knowledge_graph=kg_data,           # Knowledge graph for traversal
    use_graph_expansion=True,           # Enable graph traversal
    max_expansion_hops=2,               # Traverse up to 2 hops
    hybrid_alpha=0.6                    # 60% weight on graph, 40% on vector
)

print("AgentContext initialized with hybrid retrieval.")
print(f"  - Graph expansion: Enabled")
print(f"  - Max expansion hops: 2")
print(f"  - Hybrid alpha: 0.6 (60% graph, 40% vector)")

print(f"\nGraphRAG Synthesis Complete:")
print(f"  - Entities: {len(kg_data.get('entities', []))}")
print(f"  - Relationships: {len(kg_data.get('relationships', []))}")
print(f"  - Ready for hybrid retrieval")


AgentContext initialized with hybrid retrieval.
  - Graph expansion: Enabled
  - Max expansion hops: 2
  - Hybrid alpha: 0.6 (60% graph, 40% vector)

GraphRAG Synthesis Complete:
  - Entities: 24
  - Relationships: 27
  - Ready for hybrid retrieval


In [27]:
texts_to_embed

["The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.",
 'The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.',
 "Critics say the proposal could see even younger children becoming offenders as well as breaching children's rights.",
 'Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week.',
 'The 80-year-old is accused of crimes of humanity over his bloody war on drugs that left thousands killed.',
 'Former Real Madrid manager Jose Mourinho can stifle his former side in must-win UCL match for his current club, Benfica.',
 'Coco Gauff, who lost to Elina Svitolina in the quarterfinals of the Australian Open, was filmed smashing her racket.',
 'Hamza Al-Rubaie is one of at least 17,000 children to have lost both parents in Israel’s genocidal war on Gaza.',
 'Rial hits record \u200d1,500,000 rials 

In [28]:
chunks_to_embed

[Chunk(text="The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.", start_index=0, end_index=101, metadata={'method': 'entity_aware', 'chunk_size': 101, 'entity_count': 3, 'entities': [Entity(text='US', label='GPE', start_char=4, end_char=6, confidence=1.0, metadata={'extraction_method': 'ml', 'model': 'en_core_web_sm', 'lemma': 'US', 'batch_index': 0}), Entity(text='February', label='DATE', start_char=67, end_char=75, confidence=1.0, metadata={'extraction_method': 'ml', 'model': 'en_core_web_sm', 'lemma': 'February', 'batch_index': 0}), Entity(text='Milan', label='GPE', start_char=87, end_char=92, confidence=1.0, metadata={'extraction_method': 'ml', 'model': 'en_core_web_sm', 'lemma': 'Milan', 'batch_index': 0})]}, id=None),
 Chunk(text='The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.', start_index=0, end_index=115, metadata={'method': 'entity_aware', 'chunk_size': 115,

In [29]:
from semantica.semantic_extract.providers import create_provider

user_query = "What event is the US federal agency referring to in Milan-Cortina?"

print("=" * 80)
print("QUERY:", user_query)
print("=" * 80)

print("\n--- Standard Vector Recall ---")
v_res = vs.search(user_query, limit=3)
for i, r in enumerate(v_res, 1):
    text = r.get('metadata', {}).get('content', 'No content')
    score = r.get('score', 0)
    print(f"\nResult {i} (Score: {score:.4f}):")
    print(f"  {text[:200]}...")

print("\n--- Graph Intelligence Reasoning (Hybrid Retrieval) ---")
graph_res = ctx.retrieve(user_query, max_results=3, use_graph=True, expand_graph=True)

for i, res in enumerate(graph_res, 1):
    print(f"\nResult {i}:")
    print(f"  Content: {res.get('content', '')[:200]}...")
    print(f"  Score: {res.get('score', 0):.4f}")
    if res.get('related_entities'):
        print(f"  Multi-hop connections: {len(res['related_entities'])} entities")
        for entity in res['related_entities'][:3]:
            print(f"     - {entity.get('name', entity.get('content', 'Unknown'))}")
    if res.get('related_relationships'):
        print(f"  Related relationships: {len(res['related_relationships'])}")

print("\n--- FINAL INTELLIGENCE SYNTHESIS (GraphRAG) ---")
context_text = "\n\n".join([r.get('content', '') for r in graph_res])
prompt = f"""Based on the following intelligence context, answer the query comprehensively.

Context:
{context_text}

Query: {user_query}

Provide a detailed analysis:"""

try:
    llm_provider = create_provider("groq", model="llama-3.3-70b-versatile")
    answer = llm_provider.generate(prompt, temperature=0.3)
    print(answer)
except Exception as e:
    print(f"\nWarning: LLM synthesis skipped: {e}")
    print("However, GraphRAG successfully retrieved multi-hop context!")


QUERY: What event is the US federal agency referring to in Milan-Cortina?

--- Standard Vector Recall ---
🔄 Semantica is embedding: Generating text embedding: What event is the US federal agency referring to i... 💾 embeddings TextEmbedder |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -
Result 1 (Score: 0.6813):
  The US federal agency says it will not play an immigration role at February's event in Milan-Cortina....

Result 2 (Score: 0.5331):
  Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week....

Result 3 (Score: 0.4780):
  The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US....

--- Graph Intelligence Reasoning (Hybrid Retrieval) ---
✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.03s Extracted: -✅ Semantica is embedding: Generated embeddin

Embedding generation failed: Text cannot be empty or whitespace-only
Using random fallback embedding



Result 1:
  Content: Milan-Cortina is a GPE....
  Score: 0.6307

Result 2:
  Content: war on drugs is a EVENT....
  Score: 0.3686

Result 3:
  Content: US is a ORG....
  Score: 0.3529

--- FINAL INTELLIGENCE SYNTHESIS (GraphRAG) ---
To answer this query comprehensively, let's break down the given context and analyze the information provided.

1. **Milan-Cortina is a GPE (Geopolitical Entity):** This indicates that Milan-Cortina refers to a location or a region that can be identified on a map. Milan-Cortina is known to be a joint bid by the Italian cities of Milan and Cortina d'Ampezzo to host the Winter Olympics. However, without more specific context, it's hard to determine if this is the exact reference here, but it gives us a geographical location.

2. **War on drugs is an EVENT:** This specifies that the "war on drugs" is being considered as a particular event or a series of events. The war on drugs is a global campaign, led by the United States, of drug prohibition, military aid,

In [30]:
context_text

'Milan-Cortina is a GPE.\n\nwar on drugs is a EVENT.\n\nUS is a ORG.'

In [31]:
texts_to_embed

["The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.",
 'The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.',
 "Critics say the proposal could see even younger children becoming offenders as well as breaching children's rights.",
 'Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week.',
 'The 80-year-old is accused of crimes of humanity over his bloody war on drugs that left thousands killed.',
 'Former Real Madrid manager Jose Mourinho can stifle his former side in must-win UCL match for his current club, Benfica.',
 'Coco Gauff, who lost to Elina Svitolina in the quarterfinals of the Australian Open, was filmed smashing her racket.',
 'Hamza Al-Rubaie is one of at least 17,000 children to have lost both parents in Israel’s genocidal war on Gaza.',
 'Rial hits record \u200d1,500,000 rials 

### Vector RAG Retrieval

Query the vector store using semantic similarity only.


In [32]:
print("--- Standard Vector Recall ---")
print("Method: Semantic similarity search only")
print("Limitation: Cannot follow entity relationships\n")

# Vector search
v_res = vs.search(user_query, limit=3)

print(f"Found {len(v_res)} results:\n")
for i, r in enumerate(v_res, 1):
    text = r.get('metadata', {}).get('content', 'No content')
    score = r.get('score', 0)
    print(f"Result {i} (Score: {score:.4f}):")
    print(f"  {text[:200]}...")
    print()


--- Standard Vector Recall ---
Method: Semantic similarity search only
Limitation: Cannot follow entity relationships

🔄 Semantica is embedding: Generating text embedding: What event is the US federal agency referring to i... 💾 embeddings TextEmbedder |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -Found 3 results:

Result 1 (Score: 0.6813):
  Former Real Madrid manager Jose Mourinho can stifle his former side in must-win UCL match for his current club, Benfica....

Result 2 (Score: 0.5331):
  Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week....

Result 3 (Score: 0.4780):
  The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US....



### GraphRAG Hybrid Retrieval

Query using hybrid retrieval that combines vector search with graph traversal.


In [33]:
print("--- Graph Intelligence Reasoning (Hybrid Retrieval) ---")
print("Method: Vector search + Graph traversal")
print("Advantage: Multi-hop reasoning across entity relationships\n")

# GraphRAG hybrid retrieval
graph_res = ctx.retrieve(
    user_query, 
    max_results=3, 
    use_graph=True,      # Enable graph-based retrieval
    expand_graph=True    # Expand to related entities
)

print(f"Found {len(graph_res)} results:\n")
for i, res in enumerate(graph_res, 1):
    print(f"Result {i}:")
    print(f"  Content: {res.get('content', '')[:200]}...")
    print(f"  Score: {res.get('score', 0):.4f}")
    
    # Show multi-hop connections (GraphRAG advantage)
    if res.get('related_entities'):
        print(f"  Multi-hop connections: {len(res['related_entities'])} entities")
        print(f"  Related entities:")
        for entity in res['related_entities'][:3]:
            entity_name = entity.get('name', entity.get('content', 'Unknown'))
            print(f"     - {entity_name}")
    print()


--- Graph Intelligence Reasoning (Hybrid Retrieval) ---
Method: Vector search + Graph traversal
Advantage: Multi-hop reasoning across entity relationships

🔄 Semantica is processing: Retrieving context for: What event is the US federal agency referring to i... 🔗 context ContextRetriever |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.01s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.01s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.01s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.02s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 emb

Embedding generation failed: Text cannot be empty or whitespace-only
Using random fallback embedding


Found 3 results:

Result 1:
  Content: Milan-Cortina is a GPE....
  Score: 0.6307

Result 2:
  Content: war on drugs is a EVENT....
  Score: 0.3686

Result 3:
  Content: US is a ORG....
  Score: 0.3529



### Generate Final Answer (GraphRAG)

Use the LLM to synthesize a comprehensive answer from retrieved context.


In [34]:
# Combine retrieved context
context_text = "\n\n".join([r.get('content', '') for r in graph_res])

# Create prompt for LLM
prompt = f"""Based on the following intelligence context, answer the query comprehensively.

Context:
{context_text}

Query: {user_query}

Provide a detailed analysis:"""

print("--- FINAL INTELLIGENCE SYNTHESIS (GraphRAG) ---")
print("Using LLM to synthesize answer from multi-hop context...\n")

try:
    llm_provider = create_provider("groq", model="llama-3.3-70b-versatile")
    answer = llm_provider.generate(prompt, temperature=0.3)
    print(answer)
except Exception as e:
    print(f"Warning: LLM synthesis skipped: {e}")
    print("\nHowever, GraphRAG successfully retrieved multi-hop context!")
    print("The retrieved context above demonstrates the multi-hop connections.")


--- FINAL INTELLIGENCE SYNTHESIS (GraphRAG) ---
Using LLM to synthesize answer from multi-hop context...

To answer this query comprehensively, let's break down the given context and analyze the information step by step.

1. **Milan-Cortina is a GPE (Geopolitical Entity):** This indicates that Milan-Cortina refers to a specific geographical or political location. Milan-Cortina is known to be a region in Italy, specifically referring to the Milan-Cortina d'Ampezzo area, which has been designated to host the Winter Olympics. However, the term "GPE" suggests we are considering it in a broader geopolitical context, possibly as a venue for international events or discussions.

2. **War on drugs is an EVENT:** This statement classifies the "war on drugs" as a specific event or series of events. The war on drugs is a global campaign, led by the United States, aimed at reducing the illegal drug trade. It involves a set of policies, laws, and actions taken to combat drug trafficking and use.

3

### Conflict Detection

ConflictDetector identifies contradictory information in the knowledge graph.


In [35]:
print("\nConflict Detection")
print("-" * 50)

# Initialize ConflictDetector
conflict_detector = ConflictDetector()

# Detect conflicts in the knowledge graph
# detect_conflicts returns a list of Conflict objects, not a dictionary
conflicts = conflict_detector.detect_conflicts(kg_data)

# Filter conflicts by type
value_conflicts = [c for c in conflicts if c.conflict_type.value in ['value_conflict', 'type_conflict']]
relationship_conflicts = [c for c in conflicts if c.conflict_type.value == 'relationship_conflict']

print("Conflict detection results:")
print(f"  - Total conflicts: {len(conflicts)}")
print(f"  - Value/Type conflicts: {len(value_conflicts)}")
print(f"  - Relationship conflicts: {len(relationship_conflicts)}")

if conflicts:
    print("  Conflicts detected - review for data quality")
    # Show sample conflicts if any
    if value_conflicts:
        print(f"    Sample value conflict: {value_conflicts[0].conflict_id}")
    if relationship_conflicts:
        print(f"    Sample relationship conflict: {relationship_conflicts[0].conflict_id}")
else:
    print("  No conflicts detected - graph is consistent")

print("\nAdvanced GraphRAG features demonstrated.")



Conflict Detection
--------------------------------------------------


NameError: name 'ConflictDetector' is not defined

## 6. Visualizing the Intelligence Landscape

Seeing the 'Bridges' between disconnected events.


In [36]:
from semantica.visualization import KGVisualizer
import matplotlib.pyplot as plt

viz = KGVisualizer()
try:
    viz.visualize_network(kg_data, output="static", title="Intelligence Connectivity Map")
    plt.show()
    print("Graph visualization complete.")
except Exception as e:
    print(f"Warning: Visualization error: {e}")
    print("Graph structure:")
    print(f"  - Entities: {len(kg_data.get('entities', []))}")
    print(f"  - Relationships: {len(kg_data.get('relationships', []))}")


🔄 Semantica is visualizing: Visualizing knowledge graph network 📈 visualization KGVisualizer |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is visualizing: Visualization generated: 24 nodes, 1 edges 📈 visualization KGVisualizer |███████████████| 100.0% ETA: - Rate: - Time: 1.06s Extracted: -Graph visualization complete.


### Visualize Knowledge Graph

Create a visual representation of the knowledge graph showing entities and relationships.


In [ ]:
# Initialize KGVisualizer
viz = KGVisualizer()

print("Visualizing knowledge graph...")
print("  - Layout: Spring (force-directed)")
print("  - Title: Intelligence Connectivity Map")

try:
    viz.visualize_network(
        kg_data,
        output="static",
        title="Intelligence Connectivity Map"
    )
    plt.show()
    print("\nGraph visualization complete.")
    print(f"  - Entities: {len(kg_data.get('entities', []))}")
    print(f"  - Relationships: {len(kg_data.get('relationships', []))}")
except Exception as e:
    print(f"Warning: Visualization error: {e}")
    print("Graph structure:")
    print(f"  - Entities: {len(kg_data.get('entities', []))}")
    print(f"  - Relationships: {len(kg_data.get('relationships', []))}")


Visualizing knowledge graph...
  - Layout: Spring (force-directed)
  - Title: Intelligence Connectivity Map
🔄 Semantica is visualizing: Visualizing knowledge graph network 📈 visualization KGVisualizer |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -


Graph visualization complete.
  - Entities: 24
  - Relationships: 27


In [64]:
kg_data

{'entities': [{'id': 'us',
   'name': 'US',
   'type': 'ORG',
   'properties': {},
   'relationships': [],
   'metadata': {'provenance': {'merged_from': [{'id': 'us',
       'name': 'US',
       'source': None},
      {'id': 'us', 'name': 'US', 'source': None}],
     'merge_count': 2}},
   'merged_from': ['us', 'us'],
   'merge_strategy': 'keep_most_complete'},
  {'id': 'children',
   'name': 'children',
   'type': 'PERSON',
   'properties': {},
   'relationships': [],
   'metadata': {'provenance': {'merged_from': [{'id': 'children',
       'name': 'children',
       'source': None},
      {'id': 'children', 'name': 'children', 'source': None},
      {'id': "children's", 'name': "children's", 'source': None},
      {'id': 'currency', 'name': 'currency', 'source': None}],
     'merge_count': 4}},
   'merged_from': ['children', 'children', "children's", 'currency'],
   'merge_strategy': 'keep_most_complete'},
  {'id': 'real_madrid',
   'name': 'Real Madrid',
   'type': 'ORG',
   'propert

In [62]:
from semantica.visualization import KGVisualizer

# Create a visualizer instance
viz = KGVisualizer(layout="static", color_scheme="vibrant")

# Generate an interactive network visualization
# This returns a Plotly figure object that renders in the notebook
fig = viz.visualize_network(kg_data, output="interactive")
fig.show()

🔄 Semantica is visualizing: Visualizing knowledge graph network 📈 visualization KGVisualizer |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -

### Graph Statistics

Analyze the knowledge graph structure and quality metrics.


In [38]:
from semantica.kg import GraphAnalyzer

# Initialize GraphAnalyzer
analyzer = GraphAnalyzer()

print("\nGraphRAG Knowledge Graph Statistics")
print("-" * 50)

# Analyze graph structure
analysis = analyzer.analyze_graph(kg_data)

# Extract metrics from nested structure
metrics = analysis.get('metrics', {})
connectivity = analysis.get('connectivity', {})

print(f"Entities: {len(kg_data.get('entities', []))}")
print(f"Relationships: {len(kg_data.get('relationships', []))}")
print(f"Graph Density: {metrics.get('density', 0):.4f}")
print(f"Connected Components: {connectivity.get('connected_components', 0)}")
print(f"Average Degree: {metrics.get('avg_degree', 0):.2f}")

print("\n" + "=" * 80)
print("Comparison complete. GraphRAG demonstrates superior multi-hop reasoning capabilities.")
print("=" * 80)



GraphRAG Knowledge Graph Statistics
--------------------------------------------------
Entities: 24
Relationships: 27
Graph Density: 0.0627
Connected Components: 0
Average Degree: 1.67

Comparison complete. GraphRAG demonstrates superior multi-hop reasoning capabilities.


---

## Side-by-Side Comparison: RAG vs GraphRAG

This section demonstrates a direct comparison using the **same query** for both approaches, showing how GraphRAG generates more comprehensive answers through multi-hop reasoning.

### Comparison Methodology

1. **Same Query**: Both systems receive identical queries
2. **Context Retrieval**: Vector RAG uses semantic similarity only; GraphRAG uses hybrid retrieval
3. **Answer Generation**: Both use the same LLM to generate answers from retrieved context
4. **Analysis**: Compare answer quality, completeness, and reasoning depth


In [43]:
texts_to_embed

["The US federal agency says it will not play an immigration role at February's event in Milan-Cortina.",
 'The long-awaited deal comes as both Delhi and Brussels contend with economic and geopolitical pressure from the US.',
 "Critics say the proposal could see even younger children becoming offenders as well as breaching children's rights.",
 'Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week.',
 'The 80-year-old is accused of crimes of humanity over his bloody war on drugs that left thousands killed.',
 'Former Real Madrid manager Jose Mourinho can stifle his former side in must-win UCL match for his current club, Benfica.',
 'Coco Gauff, who lost to Elina Svitolina in the quarterfinals of the Australian Open, was filmed smashing her racket.',
 'Hamza Al-Rubaie is one of at least 17,000 children to have lost both parents in Israel’s genocidal war on Gaza.',
 'Rial hits record \u200d1,500,000 rials 

In [56]:
from semantica.semantic_extract.providers import create_provider

# Define the comparison query
comparison_query = "Trace all information related to the United States across the dataset"

# Initialize LLM provider for answer generation
llm_provider = create_provider("groq", model="llama-3.3-70b-versatile")




### Vector RAG Answer
Generate and display the answer using Vector RAG (semantic similarity only).


In [57]:
# Vector RAG: Retrieve context and generate answer
v_res = vs.search(comparison_query, limit=5)
vector_context = "\n\n".join([
    r.get('metadata', {}).get('content', 'No content') 
    for r in v_res
])

vector_prompt = f"""Based on the following context retrieved using semantic similarity search, answer the query comprehensively.

Context:
{vector_context}

Query: {comparison_query}

Provide a detailed analysis:"""

try:
    vector_answer = llm_provider.generate(vector_prompt, temperature=0.3)
except Exception as e:
    vector_answer = f"Error generating answer: {e}"

print("=" * 80)
print("VECTOR RAG ANSWER")
print("=" * 80)
print(vector_answer)


🔄 Semantica is embedding: Generating text embedding: Trace all information related to the United States... 💾 embeddings TextEmbedder |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -================================================================================
VECTOR RAG ANSWER
Based on the provided context, the information related to the United States is limited to a single mention:

"Thousands of people claim political content is being suppressed after a deal to spin off the US version of the app was signed last week."

From this statement, we can infer the following details related to the United States:

1. **Deal to spin off the US version of an app**: A deal was signed to separate the US version of an app from its parent entity. The nature of the app and the terms of the deal are not specified in the context.
2. **Allegations of political content suppression**: The deal has sparked concerns among thousands of people, who claim that political content is being suppres

### GraphRAG Answer

Generate and display the answer using GraphRAG (hybrid retrieval with graph traversal).


In [58]:
# GraphRAG: Retrieve context and generate answer
graph_res = ctx.retrieve(
    comparison_query, 
    max_results=5, 
    use_graph=True, 
    expand_graph=True,
    include_entities=True,
    include_relationships=True
)

graph_context = "\n\n".join([r.get('content', '') for r in graph_res])

graph_prompt = f"""Based on the following context retrieved using hybrid search (vector similarity + knowledge graph traversal), answer the query comprehensively.

Context:
{graph_context}

Query: {comparison_query}

Provide a detailed analysis:"""

try:
    graph_answer = llm_provider.generate(graph_prompt, temperature=0.3)
except Exception as e:
    graph_answer = f"Error generating answer: {e}"

print("=" * 80)
print("GRAPHRAG ANSWER")
print("=" * 80)
print(graph_answer)


🔄 Semantica is processing: Retrieving context for: Trace all information related to the United States... 🔗 context ContextRetriever |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.01s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.06s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.03s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbedder |███████████████| 100.0% ETA: - Rate: - Time: 0.02s Extracted: -🔄 Semantica is embedding: Using FastEmbed model... 💾 embeddings TextEmbedder |░░░░░░░░░░░░░░░| 0.0% ETA: - Rate: - Time: 0.00s Extracted: -✅ Semantica is embedding: Generated embedding (dim: 384) 💾 embeddings TextEmbed

Embedding generation failed: Text cannot be empty or whitespace-only
Using random fallback embedding


GRAPHRAG ANSWER
Based on the provided context, the query is seeking to retrieve all information related to the United States across the dataset. 

From the given context, we can identify that "US" is an abbreviation for the United States, and it is classified as an ORG (organization). 

Here's a detailed analysis of the information related to the United States:

1. **Entity Identification**: The United States is identified as "US" and is categorized as an ORG, indicating that it is an organization or a country in this context.

2. **Geopolitical Context**: Although the context does not provide explicit information about the United States' geographical location or its role in global politics, the fact that "Brussels" is mentioned as a GPE (geopolitical entity) suggests that the dataset may contain information about international relations or global events. Brussels, being the capital of Belgium and the seat of the European Union, might be related to the United States in terms of diploma

### Comparison Summary

Compare both answers side by side.


In [ ]:
# Comparison
vector_words = len(vector_answer.split()) if isinstance(vector_answer, str) and not vector_answer.startswith("Error") else 0
graph_words = len(graph_answer.split()) if isinstance(graph_answer, str) and not graph_answer.startswith("Error") else 0

print("=" * 80)
print("COMPARISON")
print("=" * 80)
if graph_words > vector_words * 1.2:
    print("✓ GraphRAG generated a more comprehensive answer")
elif graph_words < vector_words * 0.8:
    print("⚠ Vector RAG generated a longer answer")
else:
    print("→ Both approaches generated answers of similar length")
    
print("\nGraphRAG's advantage: Multi-hop reasoning discovers connections")
print("that Vector RAG cannot see, leading to more complete answers.")
print("=" * 80)


---

## Summary

This notebook demonstrated a comprehensive comparison between Standard Vector RAG and GraphRAG.

### Key Findings

1. **Vector RAG Limitations**:
   - Cannot follow entity relationships
   - Misses connections between related facts
   - Limited context expansion
   - Fails on semantic gaps

2. **GraphRAG Advantages**:
   - Multi-hop reasoning across entities
   - Relationship-aware retrieval
   - Automatic context expansion
   - Bridges semantic gaps

3. **When to Use Each**:
   - **Vector RAG**: Simple queries, single-document retrieval, fast responses
   - **GraphRAG**: Complex queries, multi-hop reasoning, relationship-aware search

### Next Steps

- Experiment with different queries to explore the differences
- Adjust hybrid_alpha to balance vector vs graph retrieval
- Explore advanced features like conflict detection and SPARQL queries
- Integrate with production systems for real-world applications


## Summary

### Key Findings

1. **Vector RAG Limitations**:
   - Only finds semantically similar text chunks
   - Cannot traverse relationships between entities
   - Fails when facts are scattered across documents
   - No multi-hop reasoning capability

2. **GraphRAG Advantages**:
   - Combines vector search with graph traversal
   - Follows relationships to find connected information
   - Bridges semantic gaps through graph structure
   - Enables multi-hop reasoning (2+ hops)
   - Provides "Chain of Evidence" for complex queries

3. **When to Use Each**:
   - **Vector RAG**: Simple fact retrieval, single-document queries, when relationships are not important
   - **GraphRAG**: Complex multi-hop queries, relationship-heavy domains, when context expansion is needed

### Conclusion

GraphRAG creates a **"Chain of Evidence"** that Vector RAG cannot see by:
- Traversing the knowledge graph structure
- Following entity relationships across documents
- Expanding context through graph connections
- Enabling multi-hop reasoning for complex questions

This makes GraphRAG particularly powerful for intelligence analysis, research, and any domain where understanding relationships is crucial.
